Modified from the [NVFlare Hello PyTorch example](https://github.com/NVIDIA/NVFlare/tree/main/examples/hello-world/hello-pt)

## Run an NVFlare Job
Now that we have defined the FedAvg controller to run our federated compute workflow on the FL server, and our client training script to receive the global models, run local training, and send the results back to the FL server, we can put everything together using NVFlare's Job API.

#### 2. Define a FedJob Recipe
 
 

In [ ]:
from server.model import FusionNet

from nvflare.app_opt.pt.recipes.fedavg import FedAvgRecipe
from nvflare.recipe import SimEnv
from nvflare.recipe import add_experiment_tracking
import torch 
n_clients = 2
num_rounds = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model = FusionNet().to(device)

recipe = FedAvgRecipe(
    name="MEM_SAN_FedCollab", # MEM - Memphis, SAN - San Diego
    min_clients=n_clients,
    num_rounds=num_rounds,
    initial_model=model,
    train_script="client.py",
)

#### 3. Add experiment tracking

In [ ]:
add_experiment_tracking(recipe, tracking_type="tensorboard")

#### 4. Run Job
Here, we run the job in a simulation environment.

In [ ]:
env = SimEnv(num_clients=n_clients)
run = recipe.execute(env)
print()
print("Job Status is:", run.get_status())
print("Result can be found in :", run.get_result())
print()

#### 5. Visualize the Training
You can use TensorBoard to show the experiment tracking curves by running

```bash
tensorboard --bind_all --logdir /tmp/nvflare/simulation/MEM_SAN_FedCollab
```
in another terminal or directly show the training curves in the next notebook cell.

In [ ]:
# asked Holger, logs are in server/simulate_job/tb_events
# they were empty because of the 2000 datapt condition.
%load_ext tensorboard
%tensorboard --bind_all --logdir /tmp/nvflare/simulation/MEM_SAN_FedCollab # also had this stuck as hello-pt